## BGP Regression Assignment 

Wenjie Cheng

October 2024

### Connect to GEE

In [1]:
import ee
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

### Define Study Area

The study area is a region around Nirobi. The map cneter is National Museum of Kenya (36.81, -1.27).

In [ ]:
Map = geemap.Map()
Map.setCenter(36.81, -1.27,7)

# the region of interest is a bounding box. Width: 1 degree. Height: 1 degree.
Nirobi_Bbox = ee.Geometry.BBox(36.31, -1.77, 37.31, -0.77)
Map.addLayer(Nirobi_Bbox, name = "Study Region")

Map

### Calculate Monthly NDVI

In [ ]:
def calculate_NDVI(month:int):
    start_date = "2020-{}-01".format(month)

    if month in [1,3,5,7,8,10,12]:
        end_date = "2020-{}-31".format(month)
    elif month in [4,6,9,11]:
        end_date = "2020-{}-30".format(month)
    elif month == 2:
        end_date = "2020-{}-29".format(month)
    else:
        print("Invalid Month")

    image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")  \
        .filterBounds(Nirobi_Bbox) \
        .filterDate(start_date, end_date) \
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 50) \
        .median()

    ndvi = (image.select("SR_B5").subtract(image.select("SR_B4")).divide(image.select("SR_B5").add(image.select("SR_B4"))))
    ndvi = ndvi.clip(Nirobi_Bbox)

    # ndvi = ee.Image(str(month))
            
    return ndvi

month = list(range(1,13))
ndvi_list = []

for m in month:
    ndvi = calculate_NDVI(m)
    ndvi_list.append(ndvi)

ndvi_collection = ee.ImageCollection.fromImages(ndvi_list)

vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

Map.addLayer(ndvi_collection.first(), vis_params, name = "ndvi")
Map
    
    

In [ ]:
ndvi_collection.size().getInfo()

### Get SRTM

In [ ]:
SRTM  = ee.Image('USGS/SRTMGL1_003').clip(Nirobi_Bbox)

stats = geemap.image_stats(SRTM,region = Nirobi_Bbox).getInfo()

SRTM_vis_params = {
    "min": stats["min"]["elevation"], 
    "max": stats["max"]["elevation"], 
    "palette": ["green", "yellow", "red"]
}

Map.addLayer(SRTM, SRTM_vis_params, name = "SRTM")
Map

### Calculate Monthly Total Precipitation

In [ ]:
def calculate_precipitation(month:int):
    start_date = "2020-{}-01".format(month)

    if month in [1,3,5,7,8,10,12]:
        end_date = "2020-{}-31".format(month)
    elif month in [4,6,9,11]:
        end_date = "2020-{}-30".format(month)
    elif month == 2:
        end_date = "2020-{}-29".format(month)
    else:
        print("Invalid Month")

    precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")  \
        .filterBounds(Nirobi_Bbox) \
        .filterDate(start_date, end_date) \
        .sum()

    precipitation = precipitation.clip(Nirobi_Bbox)
            
    return precipitation

precipitation_list = []

for m in month:
    precipitation = calculate_precipitation(m)
    precipitation_list.append(precipitation)

precipitation_collection = ee.ImageCollection.fromImages(precipitation_list)

precipitation_vis_params = {
    'min': 0,
    'max': 200,
    'palette': ['red', 'yellow', 'blue']
}

Map.addLayer(precipitation_collection.first(), precipitation_vis_params, name = "precipitation") 
Map

In [ ]:
precipitation_collection.size().getInfo()

### Random Forest Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt

def rf_regression(month):
    """Sample 1000 points from ndvi,SRTM and precipitation
    develop a random forest model for predicting NDVI values based on  elevation and precipitation
    """
    ndvi = calculate_NDVI(month)
    precipitation = calculate_precipitation(month)

    # sample points from SRTM
    points = SRTM.sample(
        region = Nirobi_Bbox, 
        scale = 30, 
        numPixels = 1000, 
        geometries=True
    )

    # get ndvi data
    points = ndvi.sampleRegions(
        collection = points,
        scale = 30,
        geometries=True
    )

    # get precipitation data
    points = precipitation.sampleRegions(
        collection = points,
        scale = 5566,
        geometries=True
    )

    #convert to pandas dataframe
    df = geemap.ee_to_df(points)

    # split dataset
    Xtrain, Xtest, ytrain, ytest = train_test_split(df[["precipitation","elevation"]], df["SR_B5"], test_size=0.3, random_state=1)

    
    scaler = StandardScaler()
    scaler.fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)

    # Random Forest Regression
    reg = RandomForestRegressor(n_estimators = 500)

    # train model
    reg.fit(Xtrain, ytrain)

    # predict
    rf_predict = reg.predict(Xtest)

    RMSE = root_mean_squared_error(ytest, rf_predict)
    print("RMSE score of Month {} is".format(month),RMSE)

    # plot scatter 
    plt.scatter(rf_predict, ytest,c='b')
    plt.scatter(ytest, ytest,c='k')
    plt.title("Month"+str(month))
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    plt.show()


In [ ]:
# result of month 1
# rf_regression(1)

# all result (around 2 minutes)
for m in month:
    rf_regression(m)